# Import Library

In [2]:
import os
import tensorflow as tf 
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from shutil import copyfile
import matplotlib.pyplot as plt
import splitfolders

# Pengambilan Folder Dataset

In [3]:
# base_dir = '\capstone\Dataset'
base_dir = '/capstone/Dataset'

Split data

In [4]:
splitfolders.ratio(
    base_dir,
    output='fixDataset',
    ratio=(.7,0.1, 0.2)
)

Copying files: 1500 files [00:06, 249.17 files/s]


In [5]:
train_dir = 'fixDataset/train/'
val_dir = 'fixDataset/val/'
val_dir = 'fixDataset/test/'
os.listdir(train_dir)

['Angklung', 'Arumba', 'Calung', 'Jengglong', 'Kendang']

# Augmentasi Gambar

In [6]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=0.45,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    brightness_range=(0.5,1.0), 
                    shear_range=0.2, 
                    zoom_range=(0.8,1.0), 
                    fill_mode='nearest', 
                    horizontal_flip=True
                    )
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # direktori data train
    target_size=(150,150), # mengubah resolusi seluruh gambar jadi 150*150
    batch_size=32,# untuk menentukan jumlah image yang akan dimasukkan ke dalam steps training
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 1198 images belonging to 5 classes.
Found 304 images belonging to 5 classes.
Found 304 images belonging to 5 classes.


In [8]:
image, label = train_generator.next()

In [9]:
class_name = os.listdir(train_dir)

In [10]:
class_name

['Angklung', 'Arumba', 'Calung', 'Jengglong', 'Kendang']

# Transfer Learning

In [11]:
from keras.callbacks import EarlyStopping

monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=3)

In [12]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience= 2,
    verbose= 1,
    factor=0.3,
    min_lr=0.000001
)

In [13]:
from tensorflow.keras.applications import InceptionV3
model = InceptionV3(weights = 'imagenet',
                    include_top=False,
                    input_shape=(150,150,3))
model.trainable = False

In [14]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [15]:
add_model = Sequential()
add_model.add(model)
add_model.add(Flatten())
add_model.add(tf.keras.layers.Dense(1024, activation='relu'))
add_model.add(tf.keras.layers.Dense(512, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(5, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 1024)              18875392  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 1

c:\Users\ahmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [16]:
history = model.fit(train_generator,
                    epochs=15,
                    shuffle=True,
                    verbose=True,
                    validation_data=val_generator,
                    callbacks=[reduce_lr,monitor_val_acc])

Epoch 1/15
38/38 [==============================] - 89s 2s/step - loss: 0.7380 - accuracy: 0.7613 - val_loss: 0.0865 - val_accuracy: 0.9671 - lr: 1.0000e-04
Epoch 2/15
38/38 [==============================] - 92s 2s/step - loss: 0.1798 - accuracy: 0.9366 - val_loss: 0.0388 - val_accuracy: 0.9868 - lr: 1.0000e-04
Epoch 3/15
38/38 [==============================] - 92s 2s/step - loss: 0.1144 - accuracy: 0.9558 - val_loss: 0.0357 - val_accuracy: 0.9901 - lr: 1.0000e-04
Epoch 4/15
38/38 [==============================] - 78s 2s/step - loss: 0.0938 - accuracy: 0.9683 - val_loss: 0.0139 - val_accuracy: 0.9934 - lr: 1.0000e-04
Epoch 5/15
38/38 [==============================] - 102s 3s/step - loss: 0.0744 - accuracy: 0.9725 - val_loss: 0.0079 - val_accuracy: 0.9967 - lr: 1.0000e-04
Epoch 6/15
38/38 [==============================] - 81s 2s/step - loss: 0.1166 - accuracy: 0.9599 - val_loss: 0.0529 - val_accuracy: 0.9737 - lr: 1.0000e-04
Epoch 7/15
38/38 [==============================] - ETA: 

In [17]:
add_model.save('Model.h5')

In [2]:
from tkinter import *
from tkinter.filedialog import askopenfile
from keras.preprocessing import image
import matplotlib.image as mpimg
import numpy as np

In [4]:
files = askopenfile()

In [1]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

var = input()
Tk().withdraw()
filepath = askopenfilename()